In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
from functools import partial
from collections import namedtuple

import polars as pl
import altair as alt
import numpy as np
from numpy.random import multivariate_normal
from scipy.interpolate import CubicSpline

from weather.helpers.epw_read import read_epw
from weather.helpers.weather_data import PALO_ALTO_20
from weather.helpers.dataframes import filter_df_by_month
from weather.helpers.time import get_month



In [48]:
df = read_epw(PALO_ALTO_20.path)
month_filter = partial(filter_df_by_month, df, PALO_ALTO_20)
june = month_filter(6).filter(pl.col("datetime").dt.day() != 30)
assert (june["datetime"].dt.date().unique_counts().unique() == 24).all()

In [49]:
june.head()

datetime,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover (used if Horizontal IR Intensity missing),Visibility,Ceiling Height,Present Weather Observation,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
datetime[μs],f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,i64,i64,i64,f64,i64,i64,f64,f64,f64
2020-06-01 00:00:00,16.0,7.8,58,0,0,339,0,0,0,0,0,0,0,350,2.9,5,5,16.1,77777,9,160,0.0,0,88,999.0,0.0,1.0
2020-06-01 01:00:00,15.8,8.5,62,0,0,339,0,0,0,0,0,0,0,350,2.7,5,5,16.1,77777,9,170,0.0,0,88,999.0,0.0,1.0
2020-06-01 02:00:00,15.5,9.2,66,0,0,339,0,0,0,0,0,0,0,350,2.4,5,5,16.1,77777,9,179,0.0,0,88,999.0,0.0,1.0
2020-06-01 03:00:00,15.3,10.0,71,0,0,339,0,0,0,0,0,0,0,350,2.2,5,5,16.1,77777,9,189,0.0,0,88,999.0,0.0,1.0
2020-06-01 04:00:00,15.1,10.7,75,3,192,339,1,0,1,147,0,231,6,350,1.9,5,5,16.1,77777,9,200,0.0,0,88,999.0,0.0,1.0


In [173]:
alt.Chart(june).mark_line().encode(
alt.X('hours(datetime):T'),
alt.Y("Dry Bulb Temperature").scale(zero=False),
color='date(datetime):O').properties(title=get_month(june))

alt.Chart(...)

In [195]:
SampledHours = namedtuple("SampledHours", ["start", "morning", "noon", "evening", "end"])

hours = SampledHours(0, 6, 12, 18, 23) # TODO apply! 

def create_time_df(df, hours:SampledHours=hours, feature="Dry Bulb Temperature", ):
    def filter_hour(hour):
        return df.filter(pl.col("datetime").dt.hour() == hour)[feature]
    def calc_first_deriv(hour_1, hour_2):
        diff = hour_2 - hour_1
        return (df.filter(pl.col("datetime").dt.hour() == hour_2)[feature] - df.filter(pl.col("datetime").dt.hour() == hour_1)[feature])/diff

    deriv_delta = 3
    return pl.DataFrame().with_columns(
        start = filter_hour(hours.start),
        am = filter_hour(hours.morning),
        noon = filter_hour(hours.noon),
        pm = filter_hour(hours.evening),
        end = filter_hour(hours.end),
        # begin_deriv = calc_first_deriv(hours.start, hours.start + deriv_delta),
        # end_deriv = calc_first_deriv(hours.end - deriv_delta, hours.end),
    )

In [196]:
tdf = create_time_df(june).with_row_index("day in month")
tdf.head()

day in month,start,am,noon,pm,end
u32,f64,f64,f64,f64,f64
0,16.0,16.2,22.2,19.9,18.6
1,18.3,18.2,28.2,25.4,22.9
2,22.4,21.2,29.4,25.4,21.6
3,20.8,18.2,27.9,23.0,19.8
4,19.2,17.1,19.0,16.6,16.1


In [197]:
alt.Chart(tdf).transform_fold(
tdf.columns[1:6],
).mark_line().encode(
    x='day in month:Q',
    y='value:Q',
    color=alt.Color('key:N').sort(tdf.columns[1:])
)


alt.Chart(...)

In [198]:
alt.Chart(tdf).transform_fold(
tdf.columns[1:6],
).mark_bar().encode(
    x='value:Q',
    y='count()',
    row=alt.Row("key:N").sort(tdf.columns[1:])
).properties(
    width=300, height=50
)


alt.Chart(...)

In [199]:
tdfo = tdf.drop("day in month")

tdfo.mean().to_numpy().flatten()
tdfo.corr().to_numpy()

array([[1.        , 0.78259065, 0.57257922, 0.56273579, 0.49017695],
       [0.78259065, 1.        , 0.53465627, 0.54844945, 0.50046856],
       [0.57257922, 0.53465627, 1.        , 0.95322279, 0.88801313],
       [0.56273579, 0.54844945, 0.95322279, 1.        , 0.96749648],
       [0.49017695, 0.50046856, 0.88801313, 0.96749648, 1.        ]])

In [200]:
tdfo.mean().to_numpy().flatten()

array([17.45517241, 17.14137931, 23.87931034, 19.57586207, 17.8862069 ])

In [191]:
samples = multivariate_normal(tdfo.mean().to_numpy().flatten(), tdfo.corr().to_numpy(), size=30)
samples[:4]

array([[16.31257934, 15.7031034 , 22.36402435, 18.12159562, 16.54505199,
        -0.09686088,  1.10400114],
       [17.2772911 , 16.0156855 , 24.18617518, 19.86410766, 17.99472628,
        -1.35422389, -0.96373067],
       [17.80638056, 17.61667011, 25.20686857, 21.33470131, 19.35336064,
        -0.25366985, -2.30210333],
       [18.39117604, 17.15647597, 24.79833226, 20.21792667, 18.30286125,
        -1.75270264, -1.28939492]])

In [192]:
# split samples => last two are derivatives.. 
temp_samples = samples[:, :-2]
deriv_samples = samples[:, -2:]
# bc_pairs = [((1, pair[0]), (1, pair[1])) for pair in deriv_samples]
bc_pairs = [((1, -0.01), (1, -0.5)) for pair in deriv_samples]

times = np.arange(0, 24, 1)
fits = [CubicSpline(np.array(hours), sample, bc_type=bc)(times) for sample, bc in zip(temp_samples, bc_pairs)]

In [193]:
samples_df = pl.DataFrame(data=fits).insert_column(0, pl.Series("time", times)).unpivot(index="time")

alt.Chart(samples_df).mark_line().encode(
    x='time:Q',
    y=alt.Y('value:Q').scale(zero=False),
    color=alt.Color("variable:O")
)

alt.Chart(...)